In [1]:
import os

from sqlalchemy import create_engine

from nba_betting_ai.data.storage import load_games, load_teams, load_gameflow


In [2]:
postgres_user = os.environ.get('POSTGRES_USER')
postgres_password = os.environ.get('POSTGRES_PASSWORD')
postgres_host = os.environ.get('POSTGRES_HOST')
postgres_port = os.environ.get('POSTGRES_PORT')
postgres_db = os.environ.get('POSTGRES_DB')


In [3]:
postgres_conn = f'postgresql://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}'
engine = create_engine(postgres_conn)

In [4]:
df_games_raw = load_games(engine)

In [32]:
df_teams = load_teams(engine)
df_games = load_games(engine)
df_gameflow = load_gameflow(engine, game_id = list(df_games['game_id'].unique()))

In [6]:
df_gameflow[:50]

,game_id,home_score,away_score,period,period_time_remaining
645501,0012000001,1,0,1,681
645502,0012000001,2,0,1,681
645503,0012000001,3,0,1,681
645504,0012000001,3,3,1,665
645505,0012000001,5,3,1,642
645506,0012000001,7,3,1,616
645507,0012000001,7,6,1,600
645508,0012000001,7,8,1,582
645509,0012000001,7,10,1,563
645510,0012000001,7,11,1,563


In [33]:
suspicious_team_id = [50009, 15019, 12325, 15020, 15022, 12315, 15025]
suspicious_teams = df_games[df_games['team_id'].isin(suspicious_team_id)]
suspicious_teams.team_name.unique()

array([], dtype=object)

In [27]:
results = df_gameflow.groupby('game_id').tail(1)

In [28]:
results[results.home_score == results.away_score]

,game_id,home_score,away_score,period,period_time_remaining


In [34]:
df_games.wl.unique()

array(['W', 'L'], dtype=object)

In [35]:
df_gameflow[df_gameflow['game_id'] == '0022200840']

,game_id,home_score,away_score,period,period_time_remaining
355518,0022200840,3,0,1,663
355519,0022200840,3,3,1,637
355520,0022200840,6,3,1,620
355521,0022200840,6,6,1,606
355522,0022200840,9,6,1,596
...,...,...,...,...,...
355625,0022200840,93,93,4,7
355626,0022200840,94,93,4,7
355627,0022200840,95,93,4,7
355628,0022200840,95,95,4,0


In [36]:
# df_games[df_games['game_id'] == '0012400001']
df_games[df_games['game_id'] == '0022200888']

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl
4927,2022-23,1610612753,ORL,Orlando Magic,0022200888,2023-02-23,ORL vs. DET,W
4928,2022-23,1610612765,DET,Detroit Pistons,0022200888,2023-02-23,DET @ ORL,L


In [37]:
df_games[df_games['game_id'] == '0012400001']

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl
1295,2024-25,1610612738,BOS,Boston Celtics,0012400001,2024-10-04,BOS @ DEN,W
1297,2024-25,1610612743,DEN,Denver Nuggets,0012400001,2024-10-04,BOS @ DEN,L


In [44]:
mask_away = df_games['matchup'].str.contains(' vs. ')
games_id_not_away = df_games['game_id'][~mask_away]
games_with_away_team = df_games['game_id'][mask_away]
games_without_away_team = ~games_id_not_away.isin(games_with_away_team)
weird_games = games_id_not_away[games_without_away_team].values
df_gameflow[df_gameflow['game_id'].isin(weird_games)].groupby('game_id').tail(1).sort_values('game_id')

,game_id,home_score,away_score,period,period_time_remaining
80633,0012400001,103,107,4,0
80162,0012400005,130,104,4,0
79917,0012400007,125,98,4,0
61780,0022400147,98,118,4,0
25329,0022401229,110,102,4,0
25452,0022401230,111,96,4,0
23541,0062400001,81,97,4,0


In [21]:
df_games[df_games['game_id'].isin(weird_games)].groupby('game_id').tail(1)[[ 'game_id', 'team_name','matchup', 'wl']].sort_values('game_id')

,game_id,team_name,matchup,wl
1303,0012400001,Denver Nuggets,BOS @ DEN,L
1293,0012400005,Denver Nuggets,DEN @ BOS,L
1297,0012400007,Washington Wizards,WAS @ TOR,L
994,0022400147,Washington Wizards,MIA @ WAS,L
409,0022401229,Milwaukee Bucks,ATL @ MIL,W
411,0022401230,Oklahoma City Thunder,HOU @ OKC,W
381,0062400001,Milwaukee Bucks,MIL @ OKC,W


In [38]:
df_games[~df_games['team_id'].isin(df_teams['team_id'].unique())]['game_id'].unique()

array([], dtype=object)

In [14]:
df_games = load_games(engine)
nonteams = df_games[~df_games['team_id'].isin(df_teams['team_id'].unique())]['game_id'].unique()
nonteams

array(['0012400060', '0012400029', '0012400011', '0012400002',
       '0032300001', '0032300005', '0032300004', '0032300003',
       '0032300006', '0012300067', '0012300054', '0012300048',
       '0012300045', '0012300040', '0012300029', '0012300014',
       '0012300022', '0012300013', '0032200003', '0032200001',
       '0032200004', '0032200006', '0032200005', '0012200038',
       '0012200027', '0012200025', '0012200008', '0012200002',
       '0032100001', '0032100004', '0032100005', '0032100003',
       '0032100006', '0032000001'], dtype=object)

In [15]:
nonteams = df_games[~df_games['team_id'].isin(df_teams['team_id'].unique())]['game_id'].unique()
nonteams

array(['0012400060', '0012400029', '0012400011', '0012400002',
       '0032300001', '0032300005', '0032300004', '0032300003',
       '0032300006', '0012300067', '0012300054', '0012300048',
       '0012300045', '0012300040', '0012300029', '0012300014',
       '0012300022', '0012300013', '0032200003', '0032200001',
       '0032200004', '0032200006', '0032200005', '0012200038',
       '0012200027', '0012200025', '0012200008', '0012200002',
       '0032100001', '0032100004', '0032100005', '0032100003',
       '0032100006', '0032000001'], dtype=object)

In [18]:
df_games[df_games['game_id'].isin(nonteams)]

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl
1190,2024-25,15025,ULM,Ratiopharm Ulm,0012400060,2024-10-16,ULM @ POR,L
1197,2024-25,1610612757,POR,Portland Trail Blazers,0012400060,2024-10-16,POR vs. ULM,W
1257,2024-25,1610612760,OKC,Oklahoma City Thunder,0012400029,2024-10-10,OKC vs. NZB,W
1259,2024-25,15020,NZB,New Zealand Breakers,0012400029,2024-10-10,NZB @ OKC,L
1284,2024-25,1610612755,PHI,Philadelphia 76ers,0012400011,2024-10-07,PHI vs. NZB,W
...,...,...,...,...,...,...,...,...
7775,2021-22,1610616849,IAH,Team Isiah,0032100006,2022-02-18,IAH vs. BAR,L
7776,2021-22,1610616848,PAY,Team Payton,0032100005,2022-02-18,PAY @ BAR,L
7777,2021-22,1610616847,BAR,Team Barry,0032100006,2022-02-18,BAR @ IAH,W
10952,2020-21,1610616834,LBN,Team LeBron,0032000001,2021-03-07,LBN @ DRT,W


In [19]:
df_games[df_games['game_id'].isin(nonteams)]

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl
1190,2024-25,15025,ULM,Ratiopharm Ulm,0012400060,2024-10-16,ULM @ POR,L
1197,2024-25,1610612757,POR,Portland Trail Blazers,0012400060,2024-10-16,POR vs. ULM,W
1257,2024-25,1610612760,OKC,Oklahoma City Thunder,0012400029,2024-10-10,OKC vs. NZB,W
1259,2024-25,15020,NZB,New Zealand Breakers,0012400029,2024-10-10,NZB @ OKC,L
1284,2024-25,1610612755,PHI,Philadelphia 76ers,0012400011,2024-10-07,PHI vs. NZB,W
...,...,...,...,...,...,...,...,...
7775,2021-22,1610616849,IAH,Team Isiah,0032100006,2022-02-18,IAH vs. BAR,L
7776,2021-22,1610616848,PAY,Team Payton,0032100005,2022-02-18,PAY @ BAR,L
7777,2021-22,1610616847,BAR,Team Barry,0032100006,2022-02-18,BAR @ IAH,W
10952,2020-21,1610616834,LBN,Team LeBron,0032000001,2021-03-07,LBN @ DRT,W


In [48]:
# delete_games(engine, list(nonteams))
# delete_games(engine, list(weird_games))

In [32]:
df_games[df_games['game_id'].isin(df_games[df_games.wl.isna()]['game_id'])].game_id

Series([], Name: game_id, dtype: object)

In [39]:
len(df_games['team_id'].unique())

30

In [12]:
df_games

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl
0,2022-23,1610612743,DEN,Denver Nuggets,0042200405,2023-06-12,DEN vs. MIA,W
1,2022-23,1610612748,MIA,Miami Heat,0042200405,2023-06-12,MIA @ DEN,L
2,2022-23,1610612748,MIA,Miami Heat,0042200404,2023-06-09,MIA vs. DEN,L
3,2022-23,1610612743,DEN,Denver Nuggets,0042200404,2023-06-09,DEN @ MIA,W
4,2022-23,1610612743,DEN,Denver Nuggets,0042200403,2023-06-07,DEN @ MIA,W
...,...,...,...,...,...,...,...,...
6799,2024-25,1610612737,ATL,Atlanta Hawks,0022400506,2025-01-07,ATL @ UTA,W
6800,2024-25,1610612745,HOU,Houston Rockets,0022400503,2025-01-07,HOU @ WAS,W
6801,2024-25,1610612764,WAS,Washington Wizards,0022400503,2025-01-07,WAS vs. HOU,L
6802,2024-25,1610612747,LAL,Los Angeles Lakers,0022400504,2025-01-07,LAL @ DAL,L


In [40]:
weird_games_2 = ['0032300004', '0032300005']
df_games[df_games['game_id'].isin(weird_games_2)]

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl


In [41]:
df_games[df_games_raw['game_id'].isin(weird_games)].sort_values(by='game_id')


/tmp/ipykernel_768/2082755278.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_games[df_games_raw['game_id'].isin(weird_games)].sort_values(by='game_id')


,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl
1297,2024-25,1610612743,DEN,Denver Nuggets,0012400001,2024-10-04,BOS @ DEN,L
1293,2024-25,1610612746,LAC,LA Clippers,0012400004,2024-10-05,LAC vs. GSW,L
1291,2024-25,1610612764,WAS,Washington Wizards,0012400007,2024-10-06,WAS @ TOR,L
1289,2024-25,1610612749,MIL,Milwaukee Bucks,0012400008,2024-10-06,MIL @ DET,L
991,2024-25,1610612748,MIA,Miami Heat,0022400147,2024-11-02,MIA @ WAS,W
994,2024-25,1610612764,WAS,Washington Wizards,0022400147,2024-11-02,MIA @ WAS,L
408,2024-25,1610612737,ATL,Atlanta Hawks,0022401229,2024-12-14,ATL @ MIL,L
409,2024-25,1610612749,MIL,Milwaukee Bucks,0022401229,2024-12-14,ATL @ MIL,W
410,2024-25,1610612745,HOU,Houston Rockets,0022401230,2024-12-14,HOU @ OKC,L
411,2024-25,1610612760,OKC,Oklahoma City Thunder,0022401230,2024-12-14,HOU @ OKC,W


In [17]:
df_games[df_games['game_id'].isin(weird_games)]

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl
5726,2024-25,1610612760,OKC,Oklahoma City Thunder,0062400001,2024-12-17,MIL @ OKC,L
5727,2024-25,1610612749,MIL,Milwaukee Bucks,0062400001,2024-12-17,MIL @ OKC,W
5752,2024-25,1610612749,MIL,Milwaukee Bucks,0022401229,2024-12-14,ATL @ MIL,W
5753,2024-25,1610612760,OKC,Oklahoma City Thunder,0022401230,2024-12-14,HOU @ OKC,W
5754,2024-25,1610612737,ATL,Atlanta Hawks,0022401229,2024-12-14,ATL @ MIL,L
5755,2024-25,1610612745,HOU,Houston Rockets,0022401230,2024-12-14,HOU @ OKC,L
6311,2024-25,1610612764,WAS,Washington Wizards,0022400147,2024-11-02,MIA @ WAS,L
6312,2024-25,1610612748,MIA,Miami Heat,0022400147,2024-11-02,MIA @ WAS,W
6586,2024-25,1610612764,WAS,Washington Wizards,0012400007,2024-10-06,WAS @ TOR,L
6589,2024-25,1610612761,TOR,Toronto Raptors,0012400007,2024-10-06,WAS @ TOR,W


In [18]:
df_gametime_agg = (
    df_gameflow.groupby('game_id')['time_remaining']
        .agg(['min', 'max'])
        .rename(columns={'min': 'min_tr', 'max': 'max_tr'})
        .assign(game_time_span=lambda x: x['max_tr'] - x['min_tr'])
        .reset_index().sort_values('min_tr')
) 
df_gametime_agg

,game_id,min_tr,max_tr,game_time_span
0,0012100001,0,2839,2839
2640,0022300676,0,2863,2863
2641,0022300677,0,2829,2829
2642,0022300678,0,2856,2856
2643,0022300679,0,2819,2819
...,...,...,...,...
1329,0022200595,0,2857,2857
1330,0022200596,0,2860,2860
1318,0022200584,0,2838,2838
3757,0032300005,5100,5923,823
